<a href="https://colab.research.google.com/github/subjekt-iv/BigGan_AudioReactive/blob/main/BigGan_AudioReactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizador con BigGan (deep music visualizer)

Este notebook es para generar videos con BigGan (Brock et al., 2018) en particular vamos a traducir audio a vectores en el espacio latente para generar interpolaciones ente multiples clases.

(Esta es una version del repsitorio: https://github.com/msieg/deep-music-visualizer creado por msieg)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# GPU
Primer vamos a confirmar que tenemos una p100 de GPU que es la que vamos a necesitar para correr esta notebook

In [1]:
! nvidia-smi

Thu May 12 00:49:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Ahora vamos a instalar el reposoitorio


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/msieg/deep-music-visualizer.git
!cd deep-music-visualizer

fatal: destination path 'deep-music-visualizer' already exists and is not an empty directory.


In [3]:
!pip install -r /content/deep-music-visualizer/requirements.txt

  Using cached scipy-1.1.0-cp37-cp37m-manylinux1_x86_64.whl (31.2 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.2.3
    Uninstalling scipy-1.2.3:
      Successfully uninstalled scipy-1.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
jax 0.3.8 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.


In [4]:
!pip install lucid==0.3.8

In [5]:
!pip install imgaug==0.2.5

In [6]:
!pip install scipy==1.2.3

  Using cached scipy-1.2.3-cp37-cp37m-manylinux1_x86_64.whl (24.8 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


In [7]:
!pip install umap-learn==0.4.0




# Vamos a ver como funcionan los comandos.



- song 

Este comando va a indicar que cancion vamos a usar, la misma tiene que estar alojada en la memoria virtual. y se indica con su nombre y el tipo de archivo ej: "beethoven.mp3"

- resolution

Nos da opciones de resolucion. La resolucion por defecto es de 512x512 (esta es la maxima resolucion) pero esta la opcion de tener una salida de 256x256 y 128x128 pixles,
indicandolas respectivamente 256 y 128.

- duration

marca la duracion del video, por defecto la duracion es la del audio. pero podemos indicar la cantidad de segundos que tendra el video, siendo el maximo el tiempo de duracion del audio.
esto lo podemos indicar en segundos.


- pitch_sensitivity

determina que tan rapido el vector de clase (contenido tematico del video) reacciona los cambios de tono en el audio. Mientras mas grande es el valor, mayor la sensibilidad.

Rango: 1 - 299

Recomendado:  200 – 295

Por defecto: 220


- tempo_sensitivity

determina que tan rapido el vector de noise (el que da las caracteristicas posicionales y de forma dentro de las clases)
va a reaccionar a los cambion de volumen y tempo. Mientras mas grande es el valor, mayor la sensibilidad.

 Rango recomendado:  0.05 – 0.8

 Por defecto: 0.25


 - depth
 
Determina la "Profundidad" de las imagenes.
 Los numeros cercanos a 
1 tienden a tener contenido mas variado dentro de la misma clase generando espacios y algunas imprectibilidades.
 los contenidos mas cercanos a 0 se acercan mas a estructuras complejas (estructras faciales o de perros,etc.) siempre dependiendo en la clase.

Rango: 0.01 - 1

Por defecto: 1


- num_classes

este comando va a indicar el numero de clases que vamos a usar, por defecto es 12 correspondiente a la escala tonal (A, A#, B, etc.).
si usamos num_classes nuestro numero va a ser menor al maximo 12
entonces las clases se van a asignar a los tonos con mayor precencia en el audio.

- classes

si queremos visualizar clases especificas aca tenemos un indice para buscarlas (https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)
el numero de clases debe ser igual al de num_classes o 12, si no usamos num_classes. (las clases van escritas solamente en numeros sin coma y con un espacio separandolas)


Por defecto: aleatorio entre 0-999


- sort_classes_by_power

a este comando le pueden asignar 1 si quieren que las doce clases respeten el orden tonal harmonico, osea que si mi primer clase es por ejemplo 345 esta va estar asignada a A, la segunda a A#, la tercera a B, etc.

- jitter

jitter previene que los mismos vectores se repitan en musica repetitiva para que la salida de video sea mas variada. si por el contrario queremos que se repitan los mismos vectores podemos asignarle el valor 0.

Rango: 0 - 1

Por defecto:0.5

- truncation

controla el tipo de imagen que se genera segun la clase. valores cercanos a 0 dan imagenes mas reconocibles y valores mas cercanos a 1 dan lo opuesto.


Rango: 0.1 - 1

Por defecto: 1

- batch_size

esto se ultiliza para videos muy largos o muy demandantes de GPU para alivianar el proceso, reduce el procesamiento de bigGan.
Bajar a menos de 30 solo si es necesario.

Por defecto: 30

- output

este comando lo utilizamos para nombrar a nuestro archivo de salida lo hacemos con el nombre y el tipo de archivo por ej:"interpolacion larga.mp4"

Por defecto: "output.mp4"

































In [8]:
!python /content/deep-music-visualizer/visualize.py --song "/content/deep-music-visualizer/gansmelody.mp3" --pitch_sensitivity 37 --tempo_sensitivity 0.05 --jitter 0 --truncation 0.1 --depth 0.00 --num_classes 2 --classes 949 947 --batch_size 25 --output "output-shrooms.mp4"


Reading audio 


Generating input vectors 

100% 2527/2527 [00:00<00:00, 6254.62it/s]


Generating frames 

 99% 100/101 [02:54<00:01,  1.75s/it]
[MoviePy] >>>> Building video output-shrooms.mp4
[MoviePy] Writing audio in output-shroomsTEMP_MPY_wvf_snd.mp4
100% 1294/1294 [00:04<00:00, 293.05it/s]
[MoviePy] Done.
[MoviePy] Writing video output-shrooms.mp4
100% 2500/2500 [01:05<00:00, 38.27it/s]
[MoviePy] Done.
[MoviePy] >>>> Video ready: output-shrooms.mp4 



# El archivo de salida
Podemos acceder al output desde el indice de archivos de la memoria virtual de colab y descargarlo haciendo click a nuestra computadora o tranferirlo a nuestro drive desde el indice de archivos haciendo drag and drop en la carpeta de my drive o drive despues de correr la celda de autorizacion acontinuacion.

Recuerden que la memoria virtual se resetea y si no bajamos o pasamos el material al drive se *ELIMINA*

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
